In [ ]:
import requests
import pandas as pd
from getpass import getpass

In [ ]:
# Wide notebook layout and larger limits for dataframes
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 500

In [ ]:
# API key can be found in LastPass
api_key = getpass()

In [ ]:
services = pd.read_json('~/aurora/services/services.json')

In [ ]:
def text_search_request(text, limit):
    api = "https://auroraai.astest.suomi.fi/service-recommender/v1"
    url = f'{api}/text_search'
    headers = {
        'content-type': 'application/json',
        'x-api-key': api_key
    }
    params = {
        'search_text': text,
        'limit': limit         
    }
    return requests.post(url, headers=headers, json=params)

def response_to_df(response, city, pattern):
    json = response.json()
    df = pd.DataFrame(json['recommended_services'])
    df = df[['service_id', 'similarity_score']]
    df = services.merge(df, left_on='id', right_on='service_id')
    
    if city is not None:
        subset = df.municipalities.apply(lambda l: city in l)
        df = df[subset]
        
    if pattern is not None:
        subset = df.descriptions.astype(str).str.contains(pattern, case=False)
        df = df[subset]
        
    return df.sort_values('similarity_score')

def text_search(text, limit=5, city=None, pattern=None):
    response = text_search_request(text, limit)
    if not response.ok:
        raise ConnectionError(response.reason)
    df = response_to_df(response, city, pattern)
    return df

In [ ]:
df = text_search(
    'nuorten työnhaku', 
    limit=50, 
    city=None,
    pattern=None)
df